In [1]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import re

import AS_motif_analysis as AS

In [2]:
# input significant alternative splicing (Skipped exons) evants with splicing sites coodinate
mxe_sig = pd.read_csv('/home/yu/R_studio_work_dir/AS_further_analysis/output/MXE_sig.csv')
print(mxe_sig.iloc[:5,])

     ID           GeneID  geneSymbol    chr strand  X1stExonStart_0base  \
0   445  ENSG00000106133     NSUN5P2   chr7      -             72952151   
1   947  ENSG00000137821      LRRC49  chr15      +             70872907   
2  1038  ENSG00000271383      NBPF19   chr1      +            149531690   
3  3556  ENSG00000183889  AC138969.4  chr16      +             16335860   
4  3235  ENSG00000141376       BCAS3  chr17      +             61380505   

   X1stExonEnd  X2ndExonStart_0base  X2ndExonEnd  upstreamES     ...       \
0     72952295             72954358     72954479    72950057     ...        
1     70873223             70895848     70895936    70853238     ...        
2    149531742            149536445    149536497   149530798     ...        
3     16335992             16336051     16336137    16333646     ...        
4     61380571             61388617     61388708    61368326     ...        

            FDR  IncLevelDifference  IncLevel1-1  IncLevel1-2 IncLevel1-3  \
0  1.5942

In [7]:
## Calculate the length of upstream,dowstream exon and intron, and flanked exon, 
## and calcualte the genome coodinates in order to download from NCBI
mxe_sig_as_coord_len = pd.DataFrame(columns = ['ID','GeneID','geneSymbol','chr','strand',
                                              'seq_1_in_co1','seq_1_in_co2','seq_1_in_co3','seq_1_in_co4',
                                              'seq_2_in_co1','seq_2_in_co2','seq_2_in_co3','seq_2_in_co4',
                                              'seq_1_ex_co1','seq_1_ex_co2','seq_1_ex_co3','seq_1_ex_co4',
                                              'seq_2_ex_co1','seq_2_ex_co2','seq_2_ex_co3','seq_2_ex_co4'])

for i in range(0,len(mxe_sig)):
    
    if mxe_sig['strand'][i] == '+':
        mxe_sig_as_coord_len.loc[i,'seq_1_in_co2'] = mxe_sig.loc[i,'upstreamEE'].copy()
        mxe_sig_as_coord_len.loc[i,'seq_1_in_co3'] = mxe_sig.loc[i,'upstreamEE'].copy() + 1
        mxe_sig_as_coord_len.loc[i,'seq_2_in_co2'] = mxe_sig.loc[i,'X1stExonStart_0base'].copy() - 1
        mxe_sig_as_coord_len.loc[i,'seq_2_in_co3'] = mxe_sig.loc[i,'X1stExonStart_0base'].copy()

        mxe_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = mxe_sig.loc[i,'upstreamEE'].copy()
        mxe_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = mxe_sig.loc[i,'upstreamEE'].copy() + 1
        mxe_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = mxe_sig.loc[i,'X2ndExonStart_0base'].copy() - 1
        mxe_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = mxe_sig.loc[i,'X2ndExonStart_0base'].copy()
  
        if (mxe_sig.loc[i,'upstreamEE'] - mxe_sig.loc[i,'upstreamES'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co1'] = mxe_sig.loc[i,'upstreamEE'].copy() - 150 + 1
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = mxe_sig.loc[i,'upstreamEE'].copy() - 150 + 1                     
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co1'] = mxe_sig.loc[i,'upstreamES']
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = mxe_sig.loc[i,'upstreamES']
                                                      
        if (mxe_sig.loc[i,'X1stExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1) >= 300:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co4'] = mxe_sig.loc[i,'upstreamEE'] + 150
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co1'] = mxe_sig.loc[i,'X1stExonStart_0base'] - 150
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co4'] = mxe_sig.loc[i,'upstreamEE'] + int((mxe_sig.loc[i,'X1stExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1)/2)
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co1'] = mxe_sig.loc[i,'X1stExonStart_0base'] - int((mxe_sig.loc[i,'X1stExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1)/2)                      
                                   
        if (mxe_sig.loc[i,'X2ndExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1) >= 300:
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = mxe_sig.loc[i,'upstreamEE'] + 150
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = mxe_sig.loc[i,'X2ndExonStart_0base'] - 150                       
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = mxe_sig.loc[i,'upstreamEE'] + int((mxe_sig.loc[i,'X2ndExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1)/2)
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = mxe_sig.loc[i,'X2ndExonStart_0base'] - int((mxe_sig.loc[i,'X2ndExonStart_0base'] - mxe_sig.loc[i,'upstreamEE'] - 1)/2)
        
        if (mxe_sig.loc[i,'X1stExonEnd'] - mxe_sig.loc[i,'X1stExonStart_0base'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co4'] = mxe_sig.loc[i,'X1stExonStart_0base'] + 150 -1
        else:
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co4'] = mxe_sig.loc[i,'X1stExonEnd']
        
        if (mxe_sig.loc[i,'X2ndExonEnd'] - mxe_sig.loc[i,'X2ndExonStart_0base'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = mxe_sig.loc[i,'X2ndExonStart_0base'] + 150 -1
        else:
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = mxe_sig.loc[i,'X2ndExonEnd']
            
    else:
        mxe_sig_as_coord_len.loc[i,'seq_1_in_co2'] = mxe_sig.loc[i,'downstreamES'].copy()
        mxe_sig_as_coord_len.loc[i,'seq_1_in_co3'] = mxe_sig.loc[i,'downstreamES'].copy() - 1
        mxe_sig_as_coord_len.loc[i,'seq_2_in_co2'] = mxe_sig.loc[i,'X2ndExonEnd'].copy() + 1
        mxe_sig_as_coord_len.loc[i,'seq_2_in_co3'] = mxe_sig.loc[i,'X2ndExonEnd'].copy()

        mxe_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = mxe_sig.loc[i,'downstreamES'].copy()
        mxe_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = mxe_sig.loc[i,'downstreamES'].copy() - 1
        mxe_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = mxe_sig.loc[i,'X1stExonEnd'].copy() + 1
        mxe_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = mxe_sig.loc[i,'X1stExonEnd'].copy()
        
        if (mxe_sig.loc[i,'downstreamEE'] - mxe_sig.loc[i,'downstreamES'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co1'] = mxe_sig.loc[i,'downstreamES'].copy() + 150 - 1
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = mxe_sig.loc[i,'downstreamES'].copy() + 150 - 1        
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co1'] = mxe_sig.loc[i,'downstreamEE']
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = mxe_sig.loc[i,'downstreamEE']
            
            
        if (mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X2ndExonEnd'] - 1) >= 300:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co4'] = mxe_sig.loc[i,'downstreamES'] - 150
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co1'] = mxe_sig.loc[i,'X2ndExonEnd'] + 150
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_in_co4'] = mxe_sig.loc[i,'downstreamES'] - int((mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X2ndExonEnd'] + 1)/2)
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co1'] = mxe_sig.loc[i,'X2ndExonEnd'] + int((mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X2ndExonEnd'] + 1)/2)                      
                                   
        if (mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X1stExonEnd'] - 1) >= 300:
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = mxe_sig.loc[i,'downstreamES'] - 150
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = mxe_sig.loc[i,'X1stExonEnd'] + 150                       
        else:
            mxe_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = mxe_sig.loc[i,'downstreamES'] - int((mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X1stExonEnd'] - 1)/2)
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = mxe_sig.loc[i,'X1stExonEnd'] + int((mxe_sig.loc[i,'downstreamES'] - mxe_sig.loc[i,'X1stExonEnd'] - 1)/2)
        
        if (mxe_sig.loc[i,'X2ndExonEnd'] - mxe_sig.loc[i,'X2ndExonStart_0base'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co4'] = mxe_sig.loc[i,'X2ndExonEnd'] - 150 + 1
        else:
            mxe_sig_as_coord_len.loc[i,'seq_2_in_co4'] = mxe_sig.loc[i,'X2ndExonStart_0base']
        
        if (mxe_sig.loc[i,'X1stExonEnd'] - mxe_sig.loc[i,'X1stExonStart_0base'] + 1) >= 150:
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = mxe_sig.loc[i,'X1stExonEnd'] -150 +1
        else:
            mxe_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = mxe_sig.loc[i,'X1stExonStart_0base']
                                   

mxe_sig_as_coord_len.iloc[:,:5] = mxe_sig.iloc[:,:5].copy()
mxe_sig_as_coord_len

,ID,GeneID,geneSymbol,chr,strand,seq_1_in_co1,seq_1_in_co2,seq_1_in_co3,seq_1_in_co4,seq_2_in_co1,...,seq_2_in_co3,seq_2_in_co4,seq_1_ex_co1,seq_1_ex_co2,seq_1_ex_co3,seq_1_ex_co4,seq_2_ex_co1,seq_2_ex_co2,seq_2_ex_co3,seq_2_ex_co4
0,445,ENSG00000106133,NSUN5P2,chr7,-,72954717,72954624,72954623,72954551,72954552,...,72954479,72954358,72954717,72954624,72954623,72954474,72952445,72952296,72952295,72952151
1,947,ENSG00000137821,LRRC49,chr15,+,70853320,70853469,70853470,70853619,70872757,...,70872907,70873056,70853320,70853469,70853470,70853619,70895698,70895847,70895848,70895936
2,1038,ENSG00000271383,NBPF19,chr1,+,149530822,149530971,149530972,149531121,149531540,...,149531690,149531742,149530822,149530971,149530972,149531121,149536295,149536444,149536445,149536497
3,3556,ENSG00000183889,AC138969.4,chr16,+,16333735,16333884,16333885,16334034,16335710,...,16335860,16335992,16333735,16333884,16333885,16334034,16335901,16336050,16336051,16336137
4,3235,ENSG00000141376,BCAS3,chr17,+,61368345,61368494,61368495,61368644,61380355,...,61380505,61380571,61368345,61368494,61368495,61368644,61388467,61388616,61388617,61388708
5,4082,ENSG00000170919,TPT1-AS1,chr13,+,45379194,45379335,45379336,45379485,45382940,...,45383090,45383235,45379194,45379335,45379336,45379485,45389584,45389733,45389734,45389820
6,4236,ENSG00000103248,MTHFSD,chr16,-,86554751,86554644,86554643,86554494,86552442,...,86552292,86552178,86554751,86554644,86554643,86554494,86552296,86552147,86552146,86551997
7,3572,ENSG00000154114,TBCEL,chr11,+,121047527,121047667,121047668,121047817,121053479,...,121053629,121053732,121047527,121047667,121047668,121047817,121054901,121055050,121055051,121055200
8,1298,ENSG00000087586,AURKA,chr20,-,56392215,56392153,56392152,56392003,56390815,...,56390665,56390555,56392215,56392153,56392152,56392003,56388934,56388785,56388784,56388686
9,5478,ENSG00000106070,GRB10,chr7,-,50756035,50755886,50755885,50755736,50732518,...,50732368,50732271,50756035,50755886,50755885,50755736,50711140,50710991,50710990,50710874
